# 二分类问题 imdb 电影评论分类

这里使用的是 imdb 电影评论.将评论分为推荐或者不推荐.


## imdb 数据集

imdb 数据集

- 50000 条评论,25000 用于训练,25000 用于验证.
- 为了方便训练,每一个整数代表一个档次,评论被转换成了整数序列.

In [15]:
from keras.datasets import imdb
(train_data, train_labels), (test_data,
                             test_labels) = imdb.load_data(num_words=10000)

加载 imdb 数据集时,参数 num_words 意味着只取词频前 10000 的单词.其他低频词被忽略.

imdb 数据集原本有 88585 个不同单词,许多词只出现一次,限制单词数量可以降低计算复杂度.


In [16]:
print(train_data[0])
print(train_labels[0])


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


train_data test_data 是单词索引组成的列表的列表. train_labels 和 test_labels 代表评论的正面(1)负面(0)


In [17]:
max([max(sequence) for sequence in train_data])

9999

限制了单词字典的总数量,索引索引最大 9999


In [18]:
word_index = imdb.get_word_index()  #将单词映射成整数的字典
reverse_word_index = dict([(value, key) \
        for (key, value) in word_index.items()])  #颠倒 value key
decoded_review = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in train_data[0]])# 解码组合单词\

print(len(word_index))
print(len(reverse_word_index))


88584
88584


这是将评论由整数数组还原成字符串的过程.

decoded_review 时,因为在数据集中 0 1 2 是为了 padding, start of sequence 和 unknow 保留的索引,所以数据集中的单词索引会比映射字典的索引大 3.


## 准备数据

电影评论有长有短,而且神经网络也无法直接处理整数序列.开始训练之前我们要将数据转换成神经网络能接受的输入.

- 既然评论有长有短,干脆都填充到最大长度.之后将列表转换成 (samples, max_length) 形状的整数张量.keras 有专门处理整数张量的层(Embedding 之后会提及)
- 或者换一种,不填充列表,反正单词限定只有 10000 个,使用一个 1d 1000 维的张量,就能表示全部的整数序列.例如整数序列 `[3,5]` 只在 3,5 取 1,其他位置取 0.在书中叫对列表进行 one-hot 编码.


In [20]:
import numpy as np


def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences),dimension))  # 初始化全 0 2d 张量
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1  #sequences 第 i 个整数序列包含的整数值在 results 中的位置设为 1
    return results


x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)


In [21]:
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

上面是手动实现的 one-hot 编码.


In [22]:
y_train = np.asarray(train_labels).astype("float32")  # 标签是直接 np 新建张量
y_test = np.asarray(test_labels).astype("float32")

评论和标签都准备完毕.
